<a href="https://colab.research.google.com/github/PraveenLiyanage/Movie-Recommender-System/blob/main/movie_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import Libraries and metadata dataset**

In [ ]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('/content/drive/MyDrive/archive/movies_metadata.csv' , low_memory=False)

# Print the first three rows
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
# Calculate mean of vote average column, c
C = metadata['vote_average'].mean()
print(C)

5.618207215134185


In [ ]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [ ]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m ]
q_movies.shape

(4555, 24)

In [ ]:
metadata.shape

(45466, 24)

### **Define a function, weighted_rating( ):**

In [ ]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
  v = x['vote_count']
  R = x['vote_average']
  # Calculation based on the IMDB formula
  return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Define a new feature 'score' and calculate its value with 'weighted_rating'
q_movies['score'] = q_movies.apply(weighted_rating, axis = 1)

In [ ]:
# Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending = False)

# Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


# **Build Plot Description Based Recommender**

In [ ]:
# Print plot overviews of the first five movies
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [ ]:
# Import TfIdVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF - IDF Vectorizer Object. Remove all english stop words such as 'the' , 'a'
tfidf = TfidfVectorizer(stop_words = 'english')

# Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

# Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [ ]:
# Array mapping from feature integer indices to feature name
tfidf.get_feature_names_out()[5000:5010]

array(['avails', 'avaks', 'avalanche', 'avalanches', 'avallone', 'avalon',
       'avant', 'avanthika', 'avanti', 'avaracious'], dtype=object)

In [ ]:
# Import linear_kernal
from sklearn.metrics.pairwise import linear_kernel

tfidf_matrix = tfidf_matrix[:30000]

# Compute the cosline similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim.shape

(30000, 30000)

In [ ]:
cosine_sim[1]

array([0.01504121, 1.        , 0.04681953, ..., 0.        , 0.        ,
       0.02303661])

In [ ]:
# Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

### **Create and define recommendation function**

In [ ]:
# Function that takes in movie title as input and outputs most simillar movies
def get_recommendations(title, cosine_sim=cosine_sim):
  # Get the index of the movie that matches title
  idx = indices[title]

  # Get the pairwise simillarity scores of all movies with that movie
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies based on the similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  # Get the scores of the 10 most simillar movies
  sim_scores = sim_scores[1:11]

  #Get the movie indices
  movie_indices = [i[0] for i in sim_scores]

  # Return the top 10 most similar movies
  return metadata['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [ ]:
get_recommendations('The Godfather')

1178      The Godfather: Part II
1914     The Godfather: Part III
23126                 Blood Ties
11297           Household Saints
10821                   Election
17729          Short Sharp Shock
26293         Beck 28 - Familjen
8653                Violent City
13177               I Am the Law
26660                   Suddenly
Name: title, dtype: object

# **Credits, Genres and Keywords Based Recommender**

In [ ]:
# Load keywords and credits
credits = pd.read_csv('/content/drive/MyDrive/archive/credits.csv')
keywords = pd.read_csv('/content/drive/MyDrive/archive/keywords.csv')

# Remove rows with bad IDs
bad_ids = [19730, 29503, 35587]
metadata = metadata.drop(index = metadata.index.intersection(bad_ids))

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on = 'id')
metadata = metadata.merge(keywords, on ='id')

In [ ]:
# Print the first two movies of your newly merged metadata
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
  metadata[feature] = metadata[feature].apply(literal_eval)

In [ ]:
# import numpy
import numpy as np

In [ ]:
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

*Return the top 3 elements or the entire list, refer the keywords*

In [ ]:
def get_list(x):
  if isinstance(x, list):
    names = [i['name'] for i in x]
    #Check if more 3 element exist. If yes, return only first three. If no, return entire list
    if len(names) > 3:
      names  = names[:3]
    return names

  # Return empty list in case of missing/malformed data
  return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
  metadata[feature] = metadata[feature].apply (get_list)

*Print first 3 features*

In [ ]:
# Print the new feature of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
  if isinstance(x, list):
    return [str.lower(i.replace(" ", "")) for i in x]
  else:
    # Check if director exist. If not, retuen empty string
    if isinstance(x, str):
      return str.lower(x.replace(" ",""))
    else:
      return ''

In [ ]:
# Apply clea_data function to the features
features = ['cast', 'keywords', 'keywords', 'genres']

for feature in features:
  metadata[feature] = metadata[feature].apply(clean_data)

In [ ]:
def create_soup(x):
  keywords = ' '.join(x['keywords']) if isinstance(x['keywords'], list) else ''
  cast = ' '.join(x['cast']) if isinstance(x['cast'], list) else ''
  director = x['director'] if isinstance(x['director'], str) else ''
  genres = ' '.join(x['genres']) if isinstance(x['genres'], list) else ''

  return keywords + ' ' + cast + ' ' + director + ' ' + genres


In [ ]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)
metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [ ]:
count_matrix.shape

(46628, 73736)

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

count_matrix = count_matrix[:10000]

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

In [ ]:
def get_recommendations(title, cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    # Make sure to handle the case where idx is out of bounds
    if idx < len(cosine_sim2):
        sim_scores = list(enumerate(cosine_sim2[idx]))
    else:
        print(f"Index {idx} is out of bounds for the cosine similarity matrix.")
        return None

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]


In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

Index 18442 is out of bounds for the cosine similarity matrix.


In [ ]:
get_recommendations('The Godfather', cosine_sim2)

1934          The Godfather: Part III
1199           The Godfather: Part II
4470    Tucker: The Man and His Dream
1186                   Apocalypse Now
4012                 Gardens of Stone
1614                    The Rainmaker
2025                    The Outsiders
3635                 The Conversation
4602                  The Cotton Club
2376            Peggy Sue Got Married
Name: title, dtype: object